# Financial Market Visualization

In [1]:
import math

from bokeh.plotting import figure, show
from bokeh.models import TickFormatter, MultiSelect, CustomJS, ColumnDataSource, HoverTool
from bokeh.layouts import row
from bokeh.palettes import Category10, Category20, Category20b, Category20c
from bokeh.util.compiler import TypeScript
from bokeh.io import output_notebook
output_notebook()

import pandas as pd
pd.set_option('display.max_rows', 100) 

import os
import numpy as np
import sqlite3 as sl
from sqlite3 import Error


Loading BokehJS ...

Create/load a database for data storage

In [2]:
db_abs_path = os.path.abspath('data.db')
db_conn = sl.connect(db_abs_path)

Read industry information

In [3]:
# create table 'industry' if not exists
db_conn.execute("""create table if not exists industry 
            (industry VARCHAR(255), link VARCHAR(255),
            PRIMARY KEY (industry))""")

# create table if not exists
db_conn.execute("""create table if not exists stock 
            (industry VARCHAR(255), name VARCHAR(255), symbol VARCHAR(255),
            PRIMARY KEY (symbol))""")

Parse industry and stock list

In [4]:
from Data.Stocks.Stock import Stock
stock = Stock(db_conn, db_abs_path)

     industry  name  symbol
0          保险  中国太保  601601
1          保险  中国平安  601318
2          保险  中国人保  601319
3          保险  新华保险  601336
4          保险  中国人寿  601628
...       ...   ...     ...
5124     专业服务  谱尼测试  300887
5125     专业服务  零点有数  301169
5126     专业服务  百普赛斯  301080
5127     专业服务   阿拉丁  688179
5128     专业服务   优宁维  301166

[5129 rows x 3 columns]
Waiting for reading stocks ...


c:\Users\yangt\anaconda3\lib\site-packages\pandas\core\generic.py:2663: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  method=method,


fail to reach 九号公司-WD: http://img1.money.126.net/data/hs/klinederc/day/times/0689009.json
Http code: 404
try get json again: 奥康国际
try get json again: 康斯特


In [5]:
industry_names = stock.industry_list

joined = pd.DataFrame(columns=["times"])
for industry_name in industry_names:
    data = pd.read_sql_query(f"SELECT * from {industry_name}", db_conn, index_col=None)
    
    values = data.loc[:, data.columns != 'times']
    data[industry_name] = values.mean(axis=1, skipna=True)
    joined = pd.merge(joined, data[['times', industry_name]], on="times", how='outer')

joined = joined.sort_values(by="times")
joined

,times,保险,玻璃玻纤,包装材料,半导体,船舶制造,采掘行业,电力行业,电网设备,电子元件,...,游戏,造纸印刷,证券,装修建材,综合行业,装修装饰,珠宝首饰,专用设备,中药,专业服务
7421,1990-12-19 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,185.300000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7422,1990-12-20 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,194.600000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7423,1990-12-21 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,204.300000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7424,1990-12-24 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,214.500000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7730,1990-12-25 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6176,2022-12-07 00:00:00,54.185000,185.496190,31.343571,121.847087,36.421,39.4915,50.085301,58.69360,214.080702,...,61.087742,29.679688,94.090000,68.626957,45.424167,23.303409,431.767222,52.912353,121.290563,69.148667
6177,2022-12-08 00:00:00,54.850000,184.660476,31.203571,119.482756,35.943,38.9500,49.501928,58.45704,215.416228,...,60.662581,29.716875,93.322708,68.607971,45.397083,23.546591,432.295556,52.776396,121.265493,68.757667
6178,2022-12-09 00:00:00,54.976667,187.073333,31.172500,121.131339,35.704,38.9815,48.928916,58.02448,211.430609,...,60.177097,29.762812,93.428750,71.589565,45.422083,23.525227,432.578889,52.558565,122.149718,69.517000
6179,2022-12-12 00:00:00,53.183333,181.680952,30.827857,123.965197,35.454,38.1805,48.066506,57.44496,207.865259,...,59.454194,29.940000,91.056667,69.720145,45.353750,23.133636,425.367222,52.621563,126.875352,69.744667


Comparison of total market values among industries in China

In [6]:
latest_values = joined.iloc[-1]
timestamp = pd.to_datetime(latest_values['times']).strftime("%Y-%m-%d")
industry_names = joined.columns.tolist()[1:]
values = latest_values.values.tolist()[1:]

p = figure(y_range=industry_names, height=1000)
p.yaxis.axis_label = '产业类型 Industries'
p.xaxis.axis_label = '收盘价 Closing Prices (人民币 RMB)'
# [i for i in range(86)]
p.hbar(y=industry_names, right=values, height=1)
show(p)

In [9]:
recent_data = joined.tail(100)
recent_data['times'] = pd.to_datetime(recent_data['times']).dt.strftime("%Y-%m-%d")

p = figure(x_range=recent_data['times'], width=800, height=500)

industry_names = recent_data.columns.tolist()[1:]
for idx, col in enumerate(industry_names):
    data = recent_data[['times', col]]
    data['name'] = col
    data.rename(columns={col: 'values'}, inplace=True)
    source = ColumnDataSource(data)     # times, values, name
    p.line(x='times', y='values', source=source, legend_label=col, name=col, 
            color=Category20b[20][idx%20])

# p.xaxis.formatter = MyFormatter()
p.xaxis.major_label_orientation = math.pi/4
p.legend.click_policy="hide"
p.legend.visible=False

hover = HoverTool(tooltips=[('Industry','@name'), ('Value', '@values')])
p.add_tools(hover)

ms = MultiSelect(title='Industries:', options=industry_names, value=industry_names, height=500)
ms.js_on_change('value', CustomJS(args=dict(legend=p.legend[0]),
                                  code="""\
    const {LegendItem} = Bokeh.require('models/annotations/legend_item');
    legend.visible = true;
    legend.items = cb_obj.value.map((v) => {
        return new LegendItem({label: {value: v},
                               renderers: [cb_obj.document.get_model_by_name(v)]});
    });
"""))

show(row(p, ms))

c:\Users\yangt\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
c:\Users\yangt\anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
c:\Users\yangt\anaconda3\lib\site-packages\pandas\core\frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/

Release all resources

In [8]:
db_conn.close()